# 目的
遺伝子学的分類に基づいた、再発の2値分類を実施する。  

In [1]:
# 基本的なライブラリ
import itertools
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# パイプライン
from imblearn.pipeline import Pipeline

# モデル
from sklearn.linear_model import Lasso
from sklearn.linear_model import ARDRegression

# 評価指標
from sklearn.metrics import mean_squared_error

# local libraries
import config
import functions

SEED = config.SEED
functions.fix_seed(SEED)


# 最大表示列数の指定（ここでは50列を指定）N
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 50)

%matplotlib inline

/home/breast-cancer-analytics/notebooks/functions.py:185: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_train: pd.Series(),
/home/breast-cancer-analytics/notebooks/functions.py:187: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  y_val: pd.Series(),


In [2]:
df_recurrenced = pd.read_pickle(
    config.INTERIM_PREPROCESSED_RECURRENCE_DIR + "/df_recurrenced4numeric.pkl"
)

classifiers = []

# データ全体での予測モデル検証

## ベースモデルの学習結果

各特徴量選択されたdfを用いて基本的な2値分類モデルを学習する

In [28]:
def validate_models(
    model_order: dict = {"original_data": []},
    input_file_path: str = None,
    output_file_path: str = None,
):

    cat_path = "/val_set"
    try:
        _X_train = pd.read_pickle(input_file_path + cat_path + "/train/X.pkl")
        _y_train = pd.read_pickle(input_file_path + cat_path + "/train/y.pkl")
        _X_val = pd.read_pickle(input_file_path + cat_path + "/val/X.pkl")
        _y_val = pd.read_pickle(input_file_path + cat_path + "/val/y.pkl")
    except:
        # pkl file dose not exist
        return

    display(_X_train.shape, _y_train.shape)

    # 学習用パイプラインの構築
    # パイプラインメソッドの関係上、オーバーサンプリングもこちらで行う
    # ※imblearnではオーバーサンプリング後のデータの保存が難しい
    # こちらはimblearnのPipelineなので注意
    _preprocess_pipeline = Pipeline(steps=model_order)
    _preprocess_pipeline.fit(_X_train, _y_train)
    _y_train_pred = _preprocess_pipeline.predict(_X_train)
    _y_val_pred = _preprocess_pipeline.predict(_X_val)
    print(mean_squared_error(_y_train, _y_train_pred, squared=False))
    print(mean_squared_error(_y_val, _y_val_pred, squared=False))

In [29]:
# 層別化する特徴量
subgroup_columns = [
    "CLAUDIN_SUBTYPE",
    # "NPI_CAT",
    # "TUMOR_CAT",
    # "LYMPH_CAT",
]
# 遺伝子発現量以外の特徴量を削除
drop_columns = [
    "CLAUDIN_SUBTYPE",
    "NPI_CAT",
    "TUMOR_CAT",
    "LYMPH_CAT",
]

# 実施したい学習の定義
model_order_dicts = [
    {
        "model_name": "ARDRegression",
        "preprocess_name": "preprocess4numeric_1",
        "model_order": [
            (
                "ARDRegression",
                ARDRegression(),
            ),
        ],
    },
    {
        "model_name": "Lasso",
        "preprocess_name": "preprocess4numeric_1",
        "model_order": [
            ("Lasso", Lasso(random_state=SEED)),
        ],
    },
]

for subgroup_column in subgroup_columns:  # 各サブグループへの適用
    print("##########" * 10)
    print("subgroup_column: ", subgroup_column)
    for subgroup in df_recurrenced[subgroup_column].unique():  # サブグループの要素毎への適用
        print("==========" * 10)
        print("subgroup: ", subgroup)
        for model_order_dict in model_order_dicts:
            print(".........." * 10)
            print("model_order_dict name: ", model_order_dict["model_name"])
            input_file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_PREPROCESSED_RECURRENCE_DIR,
                subgroup_column,
                subgroup,
                model_order_dict["preprocess_name"],
            )
            """
            output_file_path = "./{0}/{1}/{2}/{3}".format(
                config.INTERIM_MODELS_RECURRENCE_DIR,
                subgroup_column,
                subgroup,
                model_order_dict["model_name"],
            )"""
            validate_models(
                model_order=model_order_dict["model_order"],
                input_file_path=input_file_path,
            )

####################################################################################################
subgroup_column:  CLAUDIN_SUBTYPE
subgroup:  LumB
....................................................................................................
model_order_dict name:  ARDRegression


(178, 159)

(178,)

19.87581881731015
50.93123521601232
....................................................................................................
model_order_dict name:  Lasso


(178, 159)

...

(178,)

23.251475521677182
46.67407139876787
....................................................................................................
model_order_dict name:  Lasso


(178, 158)

(178,)

17.86307541914139
47.261206372504134
subgroup:  claudin-low
....................................................................................................
model_order_dict name:  ARDRegression
....................................................................................................
model_order_dict name:  Lasso
subgroup:  Basal
....................................................................................................
model_order_dict name:  ARDRegression
....................................................................................................
model_order_dict name:  Lasso
subgroup:  Normal
....................................................................................................
model_order_dict name:  ARDRegression
....................................................................................................
model_order_dict name:  Lasso
subgroup:  NC
.................................................................................